# 1.Import Depedencies

In [137]:
import pandas as pd
import numpy as np
import cv2
import time
import os
import string

# 2.Detect Centroids

In [2]:
def detect_people_bounding_box(input_path, input_name, input_ext, output_path, output_ext):
    # Video File Setup
    FILE_PATH = input_path
    FILE_NAME = input_name
    FILE_EXT = input_ext
    cap = cv2.VideoCapture(FILE_PATH + FILE_NAME + FILE_EXT)

    # Load YOLOV3
    YOLO_WEIGHTS_PATH = "yolov3.weights"
    YOLO_CONFIG_PATH = "yolov3.cfg"
    COCO_NAMES_PATH = "coco.names"
    net = cv2.dnn.readNet(YOLO_WEIGHTS_PATH, YOLO_CONFIG_PATH)
    classes = []
    with open("coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

    # Set time for FPS counter
    starting_time = time.time()
    frame_id = 0
    font = cv2.FONT_HERSHEY_PLAIN

    # Set output file (centroid)
    bounding_box = {'centroid_x' : [], 'centroid_y' : [], 'height' : []}

    # Open file in Loop
    while True:

        # Read file 
        ret, frame = cap.read()

        # Check file availability
        if(ret):

            # resize image
            frame_id += 1

            # get W,H and Depth (Channel) from image
            height, width, channels = frame.shape

            # Detecting objects on frame
            blob = cv2.dnn.blobFromImage(image = frame, scalefactor = 1/255.0, size = (416, 416), mean = (0,0,0), swapRB = True, crop=False)
            net.setInput(blob)
            outs = net.forward(output_layers)

            # Showing informations on the screen
            class_ids = []
            confidences = []
            boxes = []
            centroids = [] 

            # Loop through the detected objects
            for out in outs:
                for detection in out:

                    # Get probability and class index(argmax)
                    scores = detection[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]

                    # Set probability threshold and class id (0 = person)
                    if confidence > 0.5 and class_id == 0:
                        # Object detected
                        center_x = int(detection[0] * width)
                        center_y = int(detection[1] * height)
                        w = int(detection[2] * width)
                        h = int(detection[3] * height)

                        # Rectangle coordinates
                        x = int(center_x - w / 2)
                        y = int(center_y - h / 2)


                        # update bounding box properties
                        centroids.append([center_x,center_y])
                        boxes.append([x, y, w, h])
                        confidences.append(float(confidence))
                        class_ids.append(class_id)

            # make bounding box
            indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.8, 0.3)

            # Draw all bounding box in a frame
            for i in range(len(boxes)):
                if i in indexes:
                    # get coordinates and centroids
                    x, y, w, h = boxes[i]
                    cx, cy = centroids[i]

                    # Get label
                    label = str(classes[class_ids[i]])

                    # Draw rectangle
                    confidence = confidences[i]
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0,0,255), 2)

                    # Draw circle for centroids
                    cv2.circle(frame, (cx, cy) ,10, (0,255,0), -1)

                    # Put label to frmae
                    cv2.putText(frame, label + " " + str(round(confidence, 2)), (x, y + 30), font, 2, (0,0,255), 3)

                    # Get centroid and height for output file
                    bounding_box['centroid_x'].append(cx)
                    bounding_box['centroid_y'].append(cy)
                    bounding_box['height'].append(h)

            # Calculate FPS
            elapsed_time = time.time() - starting_time
            fps = frame_id / elapsed_time
            cv2.putText(frame, "FPS: " + str(round(fps, 2)), (10, 50), font, 4, (255, 0, 0), 3)

            # Show detected frame and the bounding boxes
            cv2.imshow("Image", frame)
            key = cv2.waitKey(1)
            if key == 27:
                break

        else:
            break


    # Export Centroids and height
    OUTPUT_PATH = output_path
    OUTPUT_NAME = input_name
    OUT_EXT = output_ext
    export_centroids = pd.DataFrame(bounding_box)
    export_centroids.to_excel(OUTPUT_PATH + FILE_NAME + OUT_EXT, index = False)

    # Close window
    cap.release()
    cv2.destroyAllWindows()


In [8]:
# video shoot 1
vid_shoot_1 = ['Abnormal 1', 'Abnormal 2', 'Abnormal 3','Abnormal 4',
                'Abnormal 5','Normal 1', 'Normal 2', 'Normal 3', 
                'Normal 4', 'Normal 5','Normal 6', 
                'Normal 7', 'Normal 8', 'Normal 9']

input_ext = ".mp4"
output_ext = ".xlsx"
vid_shoot_1_path = 'Dataset/5 FPS/Video Shoot 1/'
bounding_box_vid_shoot_1_path = "Dataset/5 FPS/Video Shoot 1/Bounding Box/"


for video_name in vid_shoot_1:
    detect_people_bounding_box(
        vid_shoot_1_path, 
        video_name, 
        input_ext, 
        bounding_box_vid_shoot_1_path,
        output_ext
    )
    

In [6]:
# video shoot 2
vid_shoot_2 = ['Abnormal 1', 'Abnormal 2', 'Abnormal 3', 
                'Abnormal 4', 'Abnormal 5','Abnormal 6', 
                'Abnormal 7', 'Abnormal 8','Normal 1', 'Normal 2', 
                'Normal 3', 'Normal 4', 'Normal 5','Normal 6', 'Normal 7']


input_ext = ".mp4"
output_ext = ".xlsx"
vid_shoot_2_path = 'Dataset/5 FPS/Video Shoot 2/'
bounding_box_vid_shoot_2_path = "Dataset/5 FPS/Video Shoot 2/Bounding Box/"


for video_name in vid_shoot_2:
    detect_people_bounding_box(
        vid_shoot_2_path, 
        video_name, 
        input_ext, 
        bounding_box_vid_shoot_2_path,
        output_ext
    )
    

In [24]:
# video Normal 
input_ext = ".mp4"
output_ext = ".xlsx"
vid_normal_path = 'Dataset/5 FPS/Normal/'
file_dir = os.listdir(vid_normal_path)
bounding_box_normal_path = "Dataset/5 FPS/Normal/Bounding Box/"
vid_normal = [name.split('.')[0] for name in file_dir]
vid_normal.remove("Bounding Box")

for video_name in vid_normal:
    detect_people_bounding_box(
        vid_normal_path, 
        video_name, 
        input_ext, 
        bounding_box_normal_path,
        output_ext
    )
    

In [30]:
# video Abnormal 
input_ext = ".mp4"
output_ext = ".xlsx"
vid_abnormal_path = 'Dataset/5 FPS/Abnormal/'
file_dir = os.listdir(vid_abnormal_path)
bounding_box_abnormal_path = "Dataset/5 FPS/Abnormal/Bounding Box/"
vid_abnormal = [name.split('.')[0] for name in file_dir]
vid_abnormal.remove("Bounding Box")


for video_name in vid_normal:
    detect_people_bounding_box(
        vid_abnormal_path, 
        video_name, 
        input_ext, 
        bounding_box_abnormal_path,
        output_ext
    )
    

# 3.Calculate L (t-1) and Tan Inverse

In [31]:
def calculate_lt_min_1(dataframe):
    h_mean = dataframe['height'].mean()
    lt_min_1 = np.array([])
    for i in range(len(df)):
        if (i < len(df) - 1) :
            x_1 = dataframe['centroid_x'].iloc[i]
            x_2 = dataframe['centroid_x'].iloc[i+1]

            y_1 = dataframe['centroid_y'].iloc[i]
            y_2 = dataframe['centroid_y'].iloc[i+1]

            res = np.sqrt((np.power((x_2 - x_1),2) + (np.power((y_2 - y_1),2)))) / h_mean
            lt_min_1 = np.append(lt_min_1, res)
        else:
            lt_min_1 = np.append(lt_min_1, np.nan)

    dataframe['l(t-1)'] = lt_min_1
    return dataframe

def calculate_tan_inv(dataframe):
    tan_inv = np.array([])
    
    for i in range(len(df)):
        if (i < len(df) - 1) :
            x_1 = dataframe['centroid_x'].iloc[i]
            x_2 = dataframe['centroid_x'].iloc[i+1]

            y_1 = dataframe['centroid_y'].iloc[i]
            y_2 = dataframe['centroid_y'].iloc[i+1]

            res = np.arctan(np.absolute(x_2 - x_1) / (np.absolute(y_2 - y_1) + 1e-6))
            tan_inv = np.append(tan_inv, res)
        else:
            tan_inv = np.append(tan_inv, np.nan)

    dataframe['tan_inv'] = tan_inv
    return dataframe

In [51]:
# Process video shoot 1
output_calculation_path = "Dataset/5 FPS/Video Shoot 1/Bounding Box/calculated result/"

for file_name in vid_shoot_1:
    df = pd.read_excel(bounding_box_vid_shoot_1_path + file_name + output_ext)
    df = calculate_lt_min_1(df)
    df = calculate_tan_inv(df)
    df.to_excel(output_calculation_path + file_name + "_calc" + output_ext, index = False)

In [52]:
# Process video shoot 2
output_calculation_path = "Dataset/5 FPS/Video Shoot 2/Bounding Box/calculated result/"

for file_name in vid_shoot_2:
    df = pd.read_excel(bounding_box_vid_shoot_2_path + file_name + output_ext)
    df = calculate_lt_min_1(df)
    df = calculate_tan_inv(df)
    df.to_excel(output_calculation_path + file_name + "_calc" + output_ext, index = False)

In [43]:
# Process video Normal
output_calculation_path = "Dataset/5 FPS/Normal/Bounding Box/calculated result/"

for file_name in vid_normal:
    df = pd.read_excel(bounding_box_normal_path + file_name + output_ext)
    df = calculate_lt_min_1(df)
    df = calculate_tan_inv(df)
    df.to_excel(output_calculation_path + file_name + "_calc" + output_ext, index = False)

In [49]:
# Process video Abnormal
output_calculation_path = "Dataset/5 FPS/Abnormal/Bounding Box/calculated result/"

for file_name in vid_abnormal:
    df = pd.read_excel(bounding_box_abnormal_path + file_name + output_ext)
    df = calculate_lt_min_1(df)
    df = calculate_tan_inv(df)
    df.to_excel(output_calculation_path + file_name + "_calc" + output_ext, index = False)

# 4.Flatten (Data Test)

In [156]:
# video shoot 1 (max version)
vid_shoot_1 = ['Abnormal 1', 'Abnormal 2', 'Abnormal 3','Abnormal 4',
                'Abnormal 5', 'Normal 5', 'Normal 8', 'Normal 9']

output_calculation_path = "Dataset/5 FPS/Video Shoot 1/Bounding Box/calculated result/"

# Find largest num of row
largest_n_row = 0
for file_name in vid_shoot_1:
    df = pd.read_excel(output_calculation_path + file_name + "_calc" + output_ext)
    df = df.drop(df.tail(1).index)
    if(len(df) > largest_n_row):
        largest_n_row = len(df)
        
# create df columns base on largest num of row
lt_cols = [('lt_' + str(i)) for i in range(largest_n_row)]
tan_inv_cols = [('tan_inv_' + str(i)) for i in range(largest_n_row)]
df_lt_cols = pd.DataFrame(columns = lt_cols, index = vid_shoot_1)
df_tan_inv_cols = pd.DataFrame(columns = tan_inv_cols, index = vid_shoot_1)

# Flatten
for file_name in vid_shoot_1:
    df = pd.read_excel(output_calculation_path + file_name + "_calc" + output_ext)
    df = df.drop(df.tail(1).index)
    
    # Update df lt columns
    lt = df['l(t-1)'].to_numpy().flatten()
    for i in range(len(lt)):
        df_lt_cols.loc[file_name][i] =  lt[i]
    
    # Update df tan inv columns
    tan_inv = df['tan_inv'].to_numpy().flatten()
    for i in range(len(tan_inv)):
        df_tan_inv_cols.loc[file_name][i] =  tan_inv[i]
        
# save 
df_res = pd.concat([df_lt_cols, df_tan_inv_cols], axis = 1)
df_res.to_excel("Dataset/5 FPS/Extracted/video shoot 1/vid_shoot_1_max.xlsx")

In [157]:
# video shoot 1 (Min Version)
vid_shoot_1 = ['Abnormal 1', 'Abnormal 2', 'Abnormal 3','Abnormal 4',
                'Abnormal 5', 'Normal 5', 'Normal 8', 'Normal 9']

output_calculation_path = "Dataset/5 FPS/Video Shoot 1/Bounding Box/calculated result/"

# Find smallest num of row
smallest_n_row = np.inf
for file_name in vid_shoot_1:
    df = pd.read_excel(output_calculation_path + file_name + "_calc" + output_ext)
    df = df.drop(df.tail(1).index)
    if(len(df) < smallest_n_row):
        smallest_n_row = len(df)
        
# create df columns base on largest num of row
lt_cols = [('lt_' + str(i)) for i in range(smallest_n_row)]
tan_inv_cols = [('tan_inv_' + str(i)) for i in range(smallest_n_row)]
df_lt_cols = pd.DataFrame(columns = lt_cols, index = vid_shoot_1)
df_tan_inv_cols = pd.DataFrame(columns = tan_inv_cols, index = vid_shoot_1)

# Flatten
for file_name in vid_shoot_1:
    df = pd.read_excel(output_calculation_path + file_name + "_calc" + output_ext)
    df = df.drop(df.tail(1).index)
    
    # Update df lt columns
    lt = df['l(t-1)'].to_numpy().flatten()
    for i in range(len(df_lt_cols.loc[file_name])):
        df_lt_cols.loc[file_name][i] =  lt[i]
    
    # Update df tan inv columns
    tan_inv = df['tan_inv'].to_numpy().flatten()
    for i in range(len(df_tan_inv_cols.loc[file_name])):
        df_tan_inv_cols.loc[file_name][i] =  tan_inv[i]
        

# save
df_res = pd.concat([df_lt_cols, df_tan_inv_cols], axis = 1)
df_res.to_excel("Dataset/5 FPS/Extracted/video shoot 1/vid_shoot_1_min.xlsx")

In [158]:
# video shoot 2 (max version)
vid_shoot_2 = ['Abnormal 1', 'Abnormal 2', 'Abnormal 3', 
                'Abnormal 4', 'Abnormal 5','Abnormal 6', 
                'Abnormal 7', 'Abnormal 8', 'Normal 2', 
                'Normal 3', 'Normal 4', 'Normal 7']

output_calculation_path = "Dataset/5 FPS/Video Shoot 2/Bounding Box/calculated result/"

# Find largest num of row
largest_n_row = 0
for file_name in vid_shoot_2:
    df = pd.read_excel(output_calculation_path + file_name + "_calc" + output_ext)
    df = df.drop(df.tail(1).index)
    if(len(df) > largest_n_row):
        largest_n_row = len(df)
        
# create df columns base on largest num of row
lt_cols = [('lt_' + str(i)) for i in range(largest_n_row)]
tan_inv_cols = [('tan_inv_' + str(i)) for i in range(largest_n_row)]
df_lt_cols = pd.DataFrame(columns = lt_cols, index = vid_shoot_2)
df_tan_inv_cols = pd.DataFrame(columns = tan_inv_cols, index = vid_shoot_2)

# Flatten 
for file_name in vid_shoot_2:
    df = pd.read_excel(output_calculation_path + file_name + "_calc" + output_ext)
    df = df.drop(df.tail(1).index)
    
    # Update df lt columns
    lt = df['l(t-1)'].to_numpy().flatten()
    for i in range(len(lt)):
        df_lt_cols.loc[file_name][i] =  lt[i]
    
    # Update df tan inv columns
    tan_inv = df['tan_inv'].to_numpy().flatten()
    for i in range(len(tan_inv)):
        df_tan_inv_cols.loc[file_name][i] =  tan_inv[i]
        

# save
df_res = pd.concat([df_lt_cols, df_tan_inv_cols], axis = 1)
df_res.to_excel("Dataset/5 FPS/Extracted/video shoot 2/vid_shoot_2_max.xlsx")

In [159]:
# video shoot 2 (Min Version)
vid_shoot_2 = ['Abnormal 1', 'Abnormal 2', 'Abnormal 3', 
                'Abnormal 4', 'Abnormal 5','Abnormal 6', 
                'Abnormal 7', 'Abnormal 8', 'Normal 2', 
                'Normal 3', 'Normal 4', 'Normal 7']

output_calculation_path = "Dataset/5 FPS/Video Shoot 2/Bounding Box/calculated result/"

# Find smallest num of row
smallest_n_row = np.inf
for file_name in vid_shoot_2:
    df = pd.read_excel(output_calculation_path + file_name + "_calc" + output_ext)
    df = df.drop(df.tail(1).index)
    if(len(df) < smallest_n_row):
        smallest_n_row = len(df)
        
# create df columns base on largest num of row
lt_cols = [('lt_' + str(i)) for i in range(smallest_n_row)]
tan_inv_cols = [('tan_inv_' + str(i)) for i in range(smallest_n_row)]
df_lt_cols = pd.DataFrame(columns = lt_cols, index = vid_shoot_2)
df_tan_inv_cols = pd.DataFrame(columns = tan_inv_cols, index = vid_shoot_2)

# Flatten
for file_name in vid_shoot_2:
    df = pd.read_excel(output_calculation_path + file_name + "_calc" + output_ext)
    df = df.drop(df.tail(1).index)
    
    # Update df lt columns
    lt = df['l(t-1)'].to_numpy().flatten()
    for i in range(len(df_lt_cols.loc[file_name])):
        df_lt_cols.loc[file_name][i] =  lt[i]
    
    # Update df tan inv columns
    tan_inv = df['tan_inv'].to_numpy().flatten()
    for i in range(len(df_tan_inv_cols.loc[file_name])):
        df_tan_inv_cols.loc[file_name][i] =  tan_inv[i]
        

df_res = pd.concat([df_lt_cols, df_tan_inv_cols], axis = 1)
df_res.to_excel("Dataset/5 FPS/Extracted/video shoot 2/vid_shoot_2_min.xlsx")

# 5.Flatten (Data Train)

In [152]:
def flat_train(files, path):
    
    # create df columns base on cut num
    n_cut = 45
    lt_cols = [('lt_' + str(i)) for i in range(n_cut)]
    tan_inv_cols = [('tan_inv_' + str(i)) for i in range(n_cut)]
    df_lt_cols = pd.DataFrame(columns = lt_cols)
    df_tan_inv_cols = pd.DataFrame(columns = tan_inv_cols)
    alphabet = list(string.ascii_lowercase)

    # observe data contains > 45 columns
    for file_name in files:
        df = pd.read_excel(path + file_name + "_calc" + output_ext)
        df = df.drop(df.tail(1).index)

        if(len(df) > n_cut):
            divided_rows = len(df) // n_cut
            cut_start = 0
            cut_end = n_cut 

            for i in range(divided_rows):
                prefix = "_"+ alphabet[i] if divided_rows > 1 else ""

                # update lt columns
                df_lt_cols.loc[file_name + prefix] = df['l(t-1)'].iloc[cut_start:cut_end].to_numpy().flatten()

                # update tan inv columns
                df_tan_inv_cols.loc[file_name + prefix] = df['tan_inv'].iloc[cut_start:cut_end].to_numpy().flatten()
                cut_start += n_cut
                cut_end += n_cut

        else:
            #  add with nan value for the remainder
            if(len(df) < n_cut):
                # update lt columns
                lt = np.concatenate([df['l(t-1)'].to_numpy().flatten(), np.full([(n_cut - len(df))], np.nan)])

                # update tan inv columns
                tan_inv = np.concatenate([df['tan_inv'].to_numpy().flatten(), np.full([(n_cut - len(df))], np.nan)])

            # else if the n cut and len df is the same then just assign the flatten
            else:
                # update lt columns
                lt = df['l(t-1)'].to_numpy().flatten()

                # update tan inv columns
                tan_inv = df['tan_inv'].to_numpy().flatten()


            df_lt_cols.loc[file_name] = lt
            df_tan_inv_cols.loc[file_name] = tan_inv

    df_res = pd.concat([df_lt_cols, df_tan_inv_cols], axis = 1)
    return df_res

In [158]:
# Video Shoot 1
vid_shoot_1 = ['Normal 1', 'Normal 2', 'Normal 3', 
                'Normal 4','Normal 6', 
                'Normal 7']
output_calculation_path = "Dataset/5 FPS/Video Shoot 1/Bounding Box/calculated result/"
vid_shoot_1_features = flat_train(vid_shoot_1, output_calculation_path)
vid_shoot_1_features

,lt_0,lt_1,lt_2,lt_3,lt_4,lt_5,lt_6,lt_7,lt_8,lt_9,...,tan_inv_35,tan_inv_36,tan_inv_37,tan_inv_38,tan_inv_39,tan_inv_40,tan_inv_41,tan_inv_42,tan_inv_43,tan_inv_44
Normal 1,0.186879,0.097098,0.154359,0.167102,0.175227,0.108351,0.150904,0.083713,0.123904,0.130382,...,0.463648,0.785398,0.588002,0.278300,1.249046,0.321751,1.570796,0.982793,1.570796,0.785398
Normal 2,0.016665,0.016028,0.034256,0.009501,0.016028,0.024998,0.015365,0.015365,0.023569,0.015365,...,1.308337,1.235158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Normal 3,0.179977,0.199567,0.123554,0.144910,0.147168,0.111995,0.069222,0.102535,0.090504,0.095504,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Normal 4,0.040001,0.017647,0.029034,0.055806,0.041061,0.035066,0.052073,0.024957,0.017647,0.054096,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Normal 6,0.009406,0.000000,0.029744,0.009406,0.047960,0.013302,0.042064,0.047029,0.050652,0.047029,...,1.570796,0.000000,1.107149,0.785398,1.418147,NaN,NaN,NaN,NaN,NaN
Normal 7,0.051608,0.055583,0.093466,0.230797,0.268559,0.051608,0.177579,0.080614,0.132180,0.134180,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [159]:
# Video Shoot 2
vid_shoot_2 = ['Normal 1', 'Normal 5','Normal 6']

output_calculation_path = "Dataset/5 FPS/Video Shoot 2/Bounding Box/calculated result/"
vid_shoot_2_features = flat_train(vid_shoot_2, output_calculation_path)
vid_shoot_2_features

,lt_0,lt_1,lt_2,lt_3,lt_4,lt_5,lt_6,lt_7,lt_8,lt_9,...,tan_inv_35,tan_inv_36,tan_inv_37,tan_inv_38,tan_inv_39,tan_inv_40,tan_inv_41,tan_inv_42,tan_inv_43,tan_inv_44
Normal 1_a,0.055753,0.083972,0.026633,0.081705,0.164682,0.043089,0.061449,0.090431,0.084959,0.115639,...,0.333244,0.146012,0.566729,0.608689,1.332960,1.107149,0.718830,1.286066,0.737815,1.019141
Normal 1_b,0.141074,0.026633,0.023290,0.164428,0.145084,0.152104,0.218904,0.111972,0.141665,0.090084,...,0.463647,0.197396,1.176005,1.494024,1.494024,0.141897,1.484058,0.218669,0.674741,1.049110
Normal 1_c,0.751786,0.743541,0.948282,0.956736,1.003281,1.009355,1.107115,0.085327,1.164342,1.268551,...,0.309199,0.178631,0.179444,0.000000,0.000000,0.075172,0.058511,0.041817,0.540419,0.162436
Normal 1_d,1.499022,1.499418,1.500635,1.500378,1.513819,0.339469,0.146799,1.981774,2.131023,0.376642,...,0.000000,0.000000,0.000000,0.000000,0.478352,0.478352,0.000000,0.166793,0.166793,0.033321
Normal 1_e,2.056494,1.965044,1.965044,1.873596,1.873457,1.749930,1.749930,1.630637,1.630637,1.376744,...,0.441793,0.441793,0.668144,0.668144,0.787615,0.588002,1.227772,0.909493,0.962033,0.962033
Normal 5_a,0.105963,0.131255,0.153505,0.149761,0.104900,0.101089,0.161037,0.073559,0.078714,0.010590,...,0.244979,0.313621,0.404892,0.398522,0.033321,0.785398,0.588003,0.334737,1.149377,1.316576
Normal 5_b,0.016744,0.055659,0.025116,0.265853,0.059198,0.144618,0.073749,0.097129,0.087325,0.103555,...,1.546991,1.570795,0.404892,1.456433,1.570796,1.390943,0.000000,1.518213,1.501142,1.512040
Normal 6_a,0.010128,0.050893,0.033303,0.044867,0.016014,0.019283,0.035267,0.042444,0.038316,0.056900,...,0.148890,0.463648,0.463648,1.501940,1.390943,1.136126,1.107149,1.512040,0.956133,1.325818
Normal 6_b,0.025446,0.028309,0.065686,0.051643,0.084168,0.132054,0.136822,0.100167,0.068177,0.076549,...,1.107148,0.876058,1.504228,0.493941,0.090660,1.313473,1.249046,1.480136,1.093451,1.428899
Normal 6_c,0.022928,0.020414,0.026435,0.025446,0.055300,0.043341,0.035808,0.075282,0.019776,0.017724,...,0.197396,1.428899,1.325818,1.249046,1.460139,1.272297,1.570796,1.405647,0.507098,1.212026


In [160]:
# Video normal
vid_normal_path = 'Dataset/5 FPS/Normal/'
file_dir = os.listdir(vid_normal_path)
vid_normal = [name.split('.')[0] for name in file_dir]
vid_normal.remove("Bounding Box")
output_calculation_path = "Dataset/5 FPS/Normal/Bounding Box/calculated result/"  
vid_normal_features = flat_train(vid_normal, output_calculation_path)
vid_normal_features

,lt_0,lt_1,lt_2,lt_3,lt_4,lt_5,lt_6,lt_7,lt_8,lt_9,...,tan_inv_35,tan_inv_36,tan_inv_37,tan_inv_38,tan_inv_39,tan_inv_40,tan_inv_41,tan_inv_42,tan_inv_43,tan_inv_44
Normal_Straight10_Outdoor,0.121310,0.137107,0.144795,0.175623,0.182518,0.092562,0.243967,0.184578,0.098624,0.276958,...,1.249046,1.424784,1.570796,1.304544,1.471128,1.373401,1.570796,1.231504,1.107149,1.435269
Normal_Straight1_Outdoor_Drizzling,0.611819,0.249250,0.253134,0.322766,0.157426,0.225138,0.152505,0.136684,0.246990,0.088091,...,1.446441,1.428899,0.785398,1.107149,1.570796,1.310194,1.405647,1.570796,1.325818,1.373401
Normal_Straight2_Outdoor_Drizzling,0.144295,0.185829,0.153006,0.278743,0.128470,0.119457,0.304432,0.121355,0.257433,0.182659,...,1.212026,1.030377,1.292497,1.071450,1.390943,1.460139,1.042722,1.292497,1.325818,0.558599
Normal_Straight3_Outdoor,0.124764,0.033939,0.042493,0.005580,0.017644,0.051441,0.024953,0.012476,0.012476,0.040235,...,1.190290,1.570796,0.785398,0.620249,1.446441,0.558599,1.325818,0.876058,0.321751,1.405648
Normal_Straight4_Outdoor,0.048644,0.323982,1.494380,0.505987,0.514059,0.000000,0.052391,0.029186,0.013758,0.009729,...,0.982793,0.885067,0.165149,1.570795,0.321750,0.404892,1.107148,0.380506,0.165149,1.570795
Normal_Straight5_Outdoor,0.158885,1.687920,0.037659,0.492316,0.023818,0.035727,0.117289,0.075319,0.093012,0.145367,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Normal_Straight6_Outdoor,0.177116,0.128871,0.159373,0.171827,0.171827,0.156625,0.201216,0.125023,0.126747,0.112211,...,1.570796,1.480136,1.487655,1.570796,1.352127,1.518213,1.460139,1.504228,1.570796,1.465919
Normal_Straight7_Outdoor,0.083510,0.238278,0.181386,0.118101,0.146336,0.109448,0.081522,0.160508,0.135870,0.199481,...,1.359703,1.494024,1.570796,1.453688,1.546991,0.960070,1.570796,1.382575,1.385448,1.267911
Normal_Straight8_Outdoor_a,0.363277,0.216436,0.007807,0.007807,0.007807,0.011041,0.050898,0.007807,0.007807,0.005521,...,0.463647,0.785398,0.785398,0.000000,0.643501,1.570796,1.570796,1.107149,0.463648,0.000000
Normal_Straight8_Outdoor_b,0.005521,0.027603,0.012345,0.056300,0.039037,0.037034,0.060978,0.011041,0.011041,0.071769,...,1.455920,1.570796,0.321751,1.012197,1.107149,1.515298,1.325818,0.982794,1.487655,0.218669


In [161]:
# Video abnormal
vid_abnormal_path = 'Dataset/5 FPS/Abnormal/'
file_dir = os.listdir(vid_abnormal_path)
vid_abnormal = [name.split('.')[0] for name in file_dir]
vid_abnormal.remove("Bounding Box")
output_calculation_path = "Dataset/5 FPS/Abnormal/Bounding Box/calculated result/"  
vid_abnormal_features = flat_train(vid_abnormal, output_calculation_path)
vid_abnormal_features

,lt_0,lt_1,lt_2,lt_3,lt_4,lt_5,lt_6,lt_7,lt_8,lt_9,...,tan_inv_35,tan_inv_36,tan_inv_37,tan_inv_38,tan_inv_39,tan_inv_40,tan_inv_41,tan_inv_42,tan_inv_43,tan_inv_44
Abnormal_Outdoor1_Drizzling_a,0.190511,0.495872,0.237901,0.291927,0.365886,0.380791,0.255136,0.107027,0.319359,0.161455,...,1.209593,1.418147,0.563316,1.428899,1.176005,0.942000,1.465919,0.732815,1.264917,1.264917
Abnormal_Outdoor1_Drizzling_b,0.058385,0.111949,0.050315,0.071156,0.105256,0.075098,0.130169,0.030039,0.069733,0.095124,...,0.486899,1.249046,1.570796,1.325818,1.275355,1.356736,1.315614,1.510264,1.570796,0.805003
Abnormal_Outdoor1_Drizzling_c,0.215513,0.092315,0.062731,0.053220,0.020642,0.091907,0.080260,0.038128,0.067356,0.090117,...,1.051650,1.523213,0.380506,1.451367,1.343997,0.439843,1.457645,1.249046,1.570796,0.000000
Abnormal_Outdoor1_Drizzling_d,0.047231,0.035401,0.030453,0.046158,0.018051,0.103212,0.028321,0.140182,0.092315,0.064115,...,1.325818,1.405648,1.570796,1.180189,1.570796,1.107149,0.507098,1.331053,1.414194,1.550799
Abnormal_Outdoor2_a,0.148264,0.041727,0.029506,0.068007,0.156478,0.063024,0.651510,0.440244,0.130502,0.177648,...,1.343997,1.570796,1.570796,1.134169,1.499489,1.504228,1.172274,1.428899,1.494024,1.396124
Abnormal_Outdoor2_b,0.081475,0.046653,0.154905,0.147713,0.074132,0.162449,0.110892,0.088517,0.168208,0.132980,...,1.325818,1.304544,1.107149,1.570796,1.249045,1.249045,0.785398,0.785398,1.570796,1.373401
Abnormal_Outdoor2_c,0.066388,0.066388,0.066796,0.052159,0.073764,0.081475,0.081141,0.074132,0.082471,0.059011,...,1.570795,0.785398,1.190290,1.504228,1.373401,1.304544,1.418147,1.471128,1.527345,1.446441
Abnormal_Outdoor2_d,0.118253,0.081475,0.059471,0.105614,0.073764,0.081475,0.104318,0.077012,0.096177,0.068007,...,1.325817,0.785398,1.570796,1.249046,1.405647,1.570796,1.325818,1.390943,1.107149,1.249046
Abnormal_Outdoor2_e,0.130711,0.069589,0.112838,0.091241,0.053701,0.155081,0.120080,0.072649,0.125616,0.128824,...,1.249046,1.515298,0.950547,1.494024,1.508377,1.499489,1.570796,1.352127,1.512040,1.405648
Abnormal_Outdoor3_a,0.528244,0.305942,0.449058,0.100915,0.247794,0.252064,0.103840,0.222729,0.185795,0.086965,...,0.000000,0.785398,0.540419,1.249045,1.190290,0.463647,1.405647,0.000000,0.321750,1.570796


In [163]:
# Save
vid_shoot_1_features.to_excel("Dataset/5 FPS/Extracted/Train/vid_shoot_1_features.xlsx")
vid_shoot_2_features.to_excel("Dataset/5 FPS/Extracted/Train/vid_shoot_2_features.xlsx")
vid_normal_features.to_excel("Dataset/5 FPS/Extracted/Train/vid_normal_features.xlsx")
vid_abnormal_features.to_excel("Dataset/5 FPS/Extracted/Train/vid_abnormal_features.xlsx")